In [5]:
import pandas as pd

test_csv_path = r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKa_basicity_data\gnn_cv\test.csv'

df_test = pd.read_csv(test_csv_path, index_col=0)

In [8]:
SMILES = df_test['Smiles'].tolist()

In [21]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

def calculate_metrics(true_values, pred_values):
    mse = round(mean_squared_error(true_values, pred_values),3)
    mae = round(mean_absolute_error(true_values, pred_values),3)
    r_score = round(r2_score(true_values, pred_values),3)

    return {"mse": mse,
            "mae": mae,
            "r^2": r_score,}

pred_pKa = [3.461677560806274,
 2.5881527423858643,
 4.142161512374878,
 4.086469612121582,
 4.392699527740478,
 4.570784664154052,
 8.319873523712157,
 8.77083963394165,
 9.65727912902832,
 7.989999370574951,
 7.061788959503174,
 10.35153003692627,
 9.162856369018554,
 8.190918769836426,
 6.678215637207031,
 10.451743888854981,
 10.010702629089355,
 10.93015785217285,
 6.292105846405029,
 10.882271881103515,
 8.586725559234619,
 8.081362323760986,
 7.354726142883301,
 11.29662769317627,
 7.010070400238037]
true_pka = df_test['pKa'].tolist()

calculate_metrics(true_values=true_pka,
                  pred_values=pred_pKa)

{'mse': 0.335, 'mae': 0.479, 'r^2': 0.944}

In [18]:
updated_pred_values = []

for index in range(len(pred_pKa)):
    print(f"True: {true_pka[index]}, pred: {pred_pKa[index]}")
    updated_pred_values.append(pred_pKa[index] - 0.2)

True: 3.55, pred: 3.461677560806274
True: 3.51, pred: 2.5881527423858643
True: 4.02, pred: 4.142161512374878
True: 4.35, pred: 4.086469612121582
True: 4.18, pred: 4.392699527740478
True: 4.61, pred: 4.570784664154052
True: 9.12, pred: 8.319873523712157
True: 9.58, pred: 8.77083963394165
True: 8.76, pred: 9.65727912902832
True: 8.1, pred: 7.989999370574951
True: 7.54, pred: 7.061788959503174
True: 9.86, pred: 10.35153003692627
True: 9.57, pred: 9.162856369018554
True: 9.01, pred: 8.190918769836426
True: 7.59, pred: 6.678215637207031
True: 10.36, pred: 10.451743888854981
True: 10.21, pred: 10.010702629089355
True: 10.5, pred: 10.93015785217285
True: 7.11, pred: 6.292105846405029
True: 11.28, pred: 10.882271881103515
True: 9.55, pred: 8.586725559234619
True: 8.97, pred: 8.081362323760986
True: 6.79, pred: 7.354726142883301
True: 11.23, pred: 11.29662769317627
True: 7.2, pred: 7.010070400238037


In [19]:
calculate_metrics(true_values=true_pka,
                  pred_values=updated_pred_values)

{'mse': 0.475, 'mae': 0.577, 'r^2': 0.921}

In [ ]:
from pkasolver.query import QueryModel
from pkasolver.ml_architecture import GINPairV1
import pickle 
import pkasolver
import torch
from os import path
from rdkit import Chem
from pkasolver.query import calculate_microstate_pka_values, draw_pka_reactions
from IPython.display import display

base_path = path.dirname(pkasolver.__file__)
# get input
print("🔮 Predicting...")

SMILES = ['CCC(F)(F)CC(O)=O',
 'OC(C1C2(CC2)C1(F)F)=O',
 'FC1(F)CC(C(O)=O)C1',
 'O=C(O)C(C1)CC21CC(F)(F)C2',
 'FC1(F)CC(C(O)=O)CCC1',
 'OC(C1C2(CC2)C1)=O',
 'FC1(F)CC(N)CCC1',
 'NC(C1)CC21CC(F)(F)C2',
 'NC1CC1',
 'NC1(CF)CCC1',
 'FC1(F)C(N)CCC1',
 'NC1CCC1',
 'FC1(F)CCC(N)CCC1',
 'FC(F)(F)C1CNCCC1',
 'CCC(F)(F)CN',
 'NCCCC(F)([H])[H]',
 'NC(C1)CC21CCC2',
 'CCCCN',
 'F[C@@H]1CNC[C@H]1F',
 'CC1(NC2)CC2C1',
 'FC([H])(F)C1CNCCC1',
 'FC(F)(F)C1CNCC1',
 'FC1(F)CNCCC1',
 'C1(NC2)CC2C1',
 'NCC([H])(F)F']

pred_pka_solver = []

for smile in SMILES:
    print(smile)
    mol = Chem.MolFromSmiles(smile)

    protonation_states = calculate_microstate_pka_values(mol, only_dimorphite=False)

    for i in range(len(protonation_states)):
        state = protonation_states[i]
        print(
            Chem.MolToSmiles(state.protonated_mol),
            Chem.MolToSmiles(state.deprotonated_mol),
        )
        print(state.pka)
        print("😀" + "--"*10 + "😀")

        pred_pka_solver.append(state.pka)

In [2]:
from pkasolver.query import QueryModel
from pkasolver.ml_architecture import GINPairV1
import pickle 
import pkasolver
import torch
from os import path
from rdkit import Chem
from pkasolver.query import calculate_microstate_pka_values, draw_pka_reactions
from IPython.display import display

# load trained model
base_path = path.dirname(pkasolver.__file__)
# get input
print("🔮 Predicting...")
smiles = "CC(CC1=CC=CC=C1)NC(=O)C(CCCCN)N"  # @param {type:"string"}
# convert from Smiles to rdkit mol
mol = Chem.MolFromSmiles(smiles)
################################################
################################################
# calculate microstate pka values
protonation_states = calculate_microstate_pka_values(mol, only_dimorphite=False)
################################################
print("📸 Visualizing ...")
# draw the micostate pka values
calc = draw_pka_reactions(protonation_states)
display(calc)

🔮 Predicting...


RuntimeError: Error(s) in loading state_dict for GINPairV1:
	Missing key(s) in state_dict: "GIN_p.convs.0.nn.lins.0.weight", "GIN_p.convs.0.nn.lins.0.bias", "GIN_p.convs.0.nn.lins.1.weight", "GIN_p.convs.0.nn.lins.1.bias", "GIN_p.convs.1.nn.lins.0.weight", "GIN_p.convs.1.nn.lins.0.bias", "GIN_p.convs.1.nn.lins.1.weight", "GIN_p.convs.1.nn.lins.1.bias", "GIN_p.convs.2.nn.lins.0.weight", "GIN_p.convs.2.nn.lins.0.bias", "GIN_p.convs.2.nn.lins.1.weight", "GIN_p.convs.2.nn.lins.1.bias", "GIN_p.convs.3.nn.lins.0.weight", "GIN_p.convs.3.nn.lins.0.bias", "GIN_p.convs.3.nn.lins.1.weight", "GIN_p.convs.3.nn.lins.1.bias", "GIN_d.convs.0.nn.lins.0.weight", "GIN_d.convs.0.nn.lins.0.bias", "GIN_d.convs.0.nn.lins.1.weight", "GIN_d.convs.0.nn.lins.1.bias", "GIN_d.convs.1.nn.lins.0.weight", "GIN_d.convs.1.nn.lins.0.bias", "GIN_d.convs.1.nn.lins.1.weight", "GIN_d.convs.1.nn.lins.1.bias", "GIN_d.convs.2.nn.lins.0.weight", "GIN_d.convs.2.nn.lins.0.bias", "GIN_d.convs.2.nn.lins.1.weight", "GIN_d.convs.2.nn

using ubuntu for prediction

In [1]:
import pandas as pd

test_set_basic = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKa_basicity_data\gnn_cv_canon_smiles\test_basic.csv')
test_set_acid = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKa_basicity_data\gnn_cv_canon_smiles\test_acid.csv')
test_set = pd.concat([test_set_basic, test_set_acid], axis=0)

print(test_set['pKa'].tolist())

[6.23, 6.06, 6.41, 7.44, 7.54, 7.75, 7.69, 8.83, 8.55, 8.98, 9.15, 9.52, 9.55, 9.67, 9.81, 9.6, 10.5, 11.27, 10.99, 3.85, 3.59, 4.18, 4.24, 4.39, 4.61, 4.38]


inference

In [4]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

def calculate_metrics(true_values, pred_values):
    mse = round(mean_squared_error(true_values, pred_values),3)
    mae = round(mean_absolute_error(true_values, pred_values),3)
    r_score = round(r2_score(true_values, pred_values),3)

    return {"mse": mse,
            "mae": mae,
            "r^2": r_score,}

pred_25_models = [6.22, 6.34, 6.54, 6.99, 7.06, 8.51, 7.97, 7.95, 7.16, 7.37, 10.08, 9.05, 8.91, 8.62, 9.03, 8.75, 10.93, 10.97, 10.91, 3.63, 3.53, 4.39, 4.86, 4.43, 4.57, 4.21]
pred_50_models = [6.34, 6.31, 6.49, 6.94, 7.0, 8.58, 8.05, 7.99, 7.18, 7.44, 10.08, 8.98, 8.84, 8.52, 9.09, 8.76, 10.91, 10.95, 10.89, 3.71, 3.5, 4.43, 4.88, 4.44, 4.59, 4.25]
pred_1_model = [6.56, 6.14, 6.39, 6.75, 7.33, 8.69, 8.41, 7.9, 6.23, 7.44, 10.27, 9.04, 8.78, 8.18, 9.47, 8.14, 10.75, 11.03, 11.0, 3.68, 3.29, 4.31, 4.73, 4.52, 4.53, 4.15]
true = [6.23, 6.06, 6.41, 7.44, 7.54, 7.75, 7.69, 8.83, 8.55, 8.98, 9.15, 9.52, 9.55, 9.67, 9.81, 9.6, 10.5, 11.27, 10.99, 3.85, 3.59, 4.18, 4.24, 4.39, 4.61, 4.38]

calculate_metrics(true_values=true,
                  pred_values=pred_50_models)

{'mse': 0.439, 'mae': 0.518, 'r^2': 0.923}

train

In [2]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

def calculate_metrics(true_values, pred_values):
    mse = round(mean_squared_error(true_values, pred_values),3)
    mae = round(mean_absolute_error(true_values, pred_values),3)
    r_score = round(r2_score(true_values, pred_values),3)

    return {"mse": mse,
            "mae": mae,
            "r^2": r_score,}


pred = [4.19, 4.76, 4.56, 3.94, 3.94, 4.74, 3.54, 3.26, 3.0, 3.93, 3.93, 4.17, 4.17, 4.42, 4.42, 3.59, 3.8, 3.95, 3.91, 4.07, 3.99, 4.06, 4.21, 4.25, 3.88, 4.19, 4.77, 4.74, 4.77, 4.77, 4.68, 4.2, 3.93, 4.17, 4.42, 3.84, 3.84, 4.11, 4.11, 4.4, 4.71, 4.71, 3.53, 8.9, 8.9, 9.31, 10.45, 9.86, 9.86, 6.18, 11.15, 11.26, 11.24, 6.21, 10.98, 10.88, 11.09, 6.22, 10.91, 9.81, 6.94, 8.05, 10.82, 7.91, 10.7, 10.81, 11.3, 6.46, 8.65, 6.8, 6.99, 6.99, 8.7, 7.0, 6.86, 6.86, 9.59, 7.24, 7.24, 5.53, 6.76, 7.97, 10.01, 8.83, 8.83, 9.22, 9.22, 9.63, 9.63, 8.07, 7.94, 6.24, 7.65, 9.34, 9.28, 9.98, 10.45, 9.92, 10.44, 8.87, 9.75, 10.32, 9.48, 10.15, 10.59, 10.04, 10.49, 10.42, 9.95, 9.64, 10.42, 7.1, 8.61, 7.61, 9.34, 7.84, 9.55, 9.55, 7.51, 9.59, 9.54, 10.32, 10.53, 10.31, 9.49, 8.0, 9.31, 7.25, 7.25, 7.57, 7.57, 9.49, 9.57]
true = [4.32, 4.85, 4.35, 3.8, 3.76, 4.5, 3.66, 3.08, 2.9, 3.83, 3.88, 4.11, 3.92, 4.18, 4.12, 3.25, 3.62, 4.02, 3.76, 3.98, 3.81, 3.78, 4.09, 4.36, 3.55, 4.17, 4.6, 4.69, 5.08, 4.75, 4.59, 4.35, 3.87, 4.26, 4.38, 3.88, 3.76, 4.18, 4.17, 4.35, 4.79, 4.83, 3.51, 8.89, 8.95, 9.45, 10.58, 9.79, 9.77, 6.52, 11.23, 11.27, 11.28, 6.56, 10.42, 10.89, 11.12, 5.96, 10.38, 9.41, 6.37, 7.72, 10.86, 8.05, 10.4, 10.53, 11.21, 6.44, 8.87, 6.88, 7.07, 7.11, 8.64, 6.79, 6.74, 6.22, 9.05, 7.01, 7.02, 5.0, 6.62, 8.1, 9.86, 8.74, 8.6, 9.17, 9.04, 9.58, 9.48, 7.99, 7.78, 5.84, 7.83, 8.96, 9.01, 9.55, 10.32, 9.75, 10.38, 8.67, 9.57, 10.05, 9.02, 10.05, 10.75, 10.19, 10.46, 10.36, 10.02, 9.74, 10.03, 7.2, 8.37, 7.45, 9.12, 8.13, 9.35, 9.57, 7.59, 9.57, 8.76, 10.52, 10.69, 10.21, 9.58, 7.95, 9.32, 7.18, 6.92, 7.76, 7.16, 9.44, 9.41]
calculate_metrics(true_values=true,
                  pred_values=pred)

{'mse': 0.051, 'mae': 0.175, 'r^2': 0.992}